In [2]:
from bs4 import BeautifulSoup
from datetime import datetime
import feedparser
import json
import os
import pandas as pd
import requests
import tabula

In [3]:
url = 'https://www.regione.sicilia.it'
feed = feedparser.parse(url+'/feed')

In [4]:
f = [field for field in feed['entries'] if 'bollettino settimanale' in field['title']]
link = f[0]['links'][0]['href']
print(link)

http://www.regione.sicilia.it/la-regione-informa/covid-bollettino-settimanale-aumento-nuovi-casi-calo-prime-dosi


In [5]:
html = requests.get(link)
soup = BeautifulSoup(html.text, 'html.parser')
links = soup.find_all('a')

In [6]:
for link in links:
    if ('.pdf' in link.get('href')):
        pdf = url+link.get('href')

print(pdf)

https://www.regione.sicilia.it/sites/default/files/2021-10/Report%20Completo%2027%20Ottobre%202021.pdf


In [7]:
r = requests.get(pdf, stream=True)
with open('./downloads/'+datetime.now().strftime("%Y%m%d")+'.pdf', 'wb') as f:
    f.write(r.content)
    print("PDF Scaricato.")

PDF Scaricato.


In [8]:
files = []
path = './downloads/'

for file in os.listdir(path):
    if (file.endswith('.pdf')):
        files.append(file)

files.sort()
file = files[0]

## Tabula

In [9]:
print('Leggo PDF...attendi...')
pdf = tabula.read_pdf_with_template(path+file, 'template_vaccini.json', pandas_options={'header': None}, multiple_tables=False, silent=True)
print('Ho letto.')

Leggo PDF...attendi...
Ho letto.


In [10]:
vax = pd.concat(pdf).reset_index(drop=True)
vax.columns = ['comune', '%vaccinati1dose', '%Immunizzati']

In [11]:
comuni = pd.DataFrame(pd.read_csv("./utilities/Elenco-comuni-siciliani.csv", converters={'pro_com_t': '{:0>6}'.format}))

In [12]:
out = pd.merge(vax, comuni, on='comune', how='inner')
out = out[['cod_prov','pro_com_t','provincia','comune','%vaccinati1dose','%Immunizzati']]

In [13]:
out["%vaccinati1dose"] = out["%vaccinati1dose"].str.replace(',','.').str.rstrip('%')
out["%Immunizzati"] = out["%Immunizzati"].str.replace(',','.').str.rstrip('%')

In [14]:
out

,cod_prov,pro_com_t,provincia,comune,%vaccinati1dose,%Immunizzati
0,84,084001,Agrigento,Agrigento,87.37,84.36
1,84,084002,Agrigento,Alessandria della Rocca,84.65,82.72
2,84,084003,Agrigento,Aragona,88.61,86.08
3,84,084004,Agrigento,Bivona,85.46,83.37
4,84,084005,Agrigento,Burgio,89.60,88.12
...,...,...,...,...,...,...
385,81,081020,Trapani,San Vito Lo Capo,74.59,68.89
386,81,081019,Trapani,Santa Ninfa,87.36,84.00
387,81,081021,Trapani,Trapani,79.17,74.29
388,81,081022,Trapani,Valderice,80.79,76.27


In [ ]:
print('Converto in CSV.')
out.to_csv('tabula-'+file[:-4]+'.csv', index=None, header=True)